In [1]:
# Dependencies
import matplotlib.pyplot as plt
import os
import sys
from nltk.corpus import stopwords
from collections import Counter
from string import punctuation
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [2]:
#Read Mails
mails_path = 'Mail_Test'

spam = []
ham = []
lines = []
file_data = []
separator = '/'
files = -1
for (path,dirname,filename) in os.walk(mails_path):
    for counter, i in enumerate(filename):
        #print(i)
        if i.endswith('.ham.txt'):
            ham_path = path + separator + i
            with open(ham_path) as f:
                try:
                    files += 1
                    #print(counter)
                    file_data.insert(counter, [[], 'ham'])
                    #print(file_data)
                    for i, line in enumerate(f):
                        if i != 0: ## Ignore Subject Line
                            words = line.split()
                            file_data[counter][0] += words
                            #lines.append(words)
                            ham += words         
                    #print(file_data)
                except:
                    pass
                    
        elif i.endswith('.spam.txt'):
            spam_path = path + separator + i
            with open(spam_path) as f:
                try:
                    files += 1
                    file_data.insert(counter, [[], 'spam'])
                    for i, line in enumerate(f):
                        if i != 0: ## Ignore Subject Line
                            words = line.split()
                            file_data[counter][0] += words
                            #lines.append(words)
                            spam += words
                except:
                    pass
#print(file_data[0], file_data[2])                    
print('Ham Folder: {}, Spam Folder: {}, Total Files: {}, Data colledcted in Files: {}'.format(len(ham), len(spam), files+1, len(file_data)))

Ham Folder: 1259046, Spam Folder: 1315130, Total Files: 10347, Data colledcted in Files: 10347


In [3]:
# Count the occurences of letters
all_count_ham = Counter(ham)
print (all_count_ham.most_common(10))

all_count_spam = Counter(spam)
print (all_count_spam.most_common(10))

[('-', 107017), ('.', 51914), ('/', 51498), (',', 44336), ('the', 31720), (':', 27089), ('to', 25919), ('ect', 21689), ('@', 17978), ('and', 15184)]
[('.', 73520), (',', 42568), ('_', 35999), ('-', 32445), ('the', 30881), ('to', 26233), ('and', 21833), ('of', 19394), (':', 16670), ('a', 16121)]


In [4]:
# Remove Junk characters and common characters
eng_stopwords = stopwords.words('english')
punctuation_marks = list(punctuation)

ham_stopwordless = list(filter(lambda x: x not in eng_stopwords and x not in punctuation_marks and x.isalpha() == True and len(x) != 1, ham))
spam_stopwordless = list(filter(lambda x: x not in eng_stopwords and x not in punctuation_marks and x.isalpha() == True and len(x) != 1, spam))

a = [[1,2,4,5], [3,5,6]]
print(list(filter(lambda x: x not in [1,6] , a)))
print('Ham Folder: {}, Spam Folder: {}, Total Files: {}'.format(len(ham_stopwordless), len(spam_stopwordless), files))

[[1, 2, 4, 5], [3, 5, 6]]
Ham Folder: 500910, Spam Folder: 587250, Total Files: 10346


In [5]:
# Count the occurences of letters
print ('---------Ham Most Common Words----------')
print (' ')
all_count_ham = Counter(ham_stopwordless)
print (all_count_ham.most_common(100))
print (' ')
print ('---------Spam Most Common Words----------')
print ('               ')
all_count_spam = Counter(spam_stopwordless)
print (all_count_spam.most_common(100))

#join lists
list_combined = ham_stopwordless + spam_stopwordless
print ('---------Most Common Words----------')
print (' ')

all_count = Counter(list_combined)
features = all_count.most_common(3000)

print(all_count.most_common(100))
#print(features[12])


---------Ham Most Common Words----------
 
[('ect', 21689), ('enron', 11106), ('hou', 10252), ('please', 4032), ('subject', 3665), ('cc', 3238), ('pm', 3201), ('com', 3125), ('gas', 3009), ('deal', 2894), ('corp', 2549), ('thanks', 2480), ('know', 2149), ('meter', 2132), ('need', 1930), ('daren', 1906), ('forwarded', 1828), ('hpl', 1806), ('would', 1780), ('let', 1602), ('new', 1539), ('attached', 1444), ('may', 1380), ('mmbtu', 1350), ('see', 1310), ('time', 1282), ('get', 1251), ('houston', 1209), ('day', 1205), ('questions', 1177), ('ena', 1169), ('business', 1168), ('sally', 1162), ('farmer', 1161), ('information', 1155), ('also', 1095), ('risk', 1068), ('lon', 1032), ('management', 1020), ('xls', 1015), ('us', 1013), ('robert', 1002), ('price', 998), ('call', 992), ('th', 971), ('one', 970), ('following', 962), ('contract', 933), ('bob', 927), ('sent', 924), ('month', 920), ('volume', 913), ('texas', 896), ('energy', 895), ('message', 887), ('meeting', 870), ('mail', 856), ('sitar

In [6]:
# Build Feature Matrix
X_features = np.zeros((files+1, 3000))
Y_features = []
print (X_features.shape)

for counter, i in enumerate(file_data):
    #print(counter, i)
    for counter_f, j in enumerate(features):
        X_features[counter][counter_f] = i[0].count(j[0])
    Y_features.insert(counter, i[1]) 


(10347, 3000)


In [7]:
# Training and Test Data Split
X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y_features, test_size = 0.25, random_state = 42)

print(X_train.shape, X_test.shape, len(Y_train), len(Y_test))

(7760, 3000) (2587, 3000) 7760 2587


In [8]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

clf_rand_forest = RandomForestClassifier(random_state = 42, min_samples_leaf = 1, min_samples_split = 5) #Obtained from GridSearchCV
clf_rand_forest.fit(X_train, Y_train)
clf_rand_forest.score(X_test, Y_test)

0.97100889060688056

In [9]:
#SVM Classifier
from sklearn.svm import SVC

clf_svc = SVC(random_state = 1)
clf_svc.fit(X_train, Y_train)
clf_svc.score(X_test, Y_test)

0.9311944337069965

In [10]:
# Gaussian Naive bayes
from sklearn.naive_bayes import GaussianNB

clf_nb = GaussianNB()
clf_nb.fit(X_train, Y_train)
clf_nb.score(X_test, Y_test)

0.90916119056822575